In [1]:
import torch

bin_offset_size = 25
bin_angle_size = 0.25 * torch.pi/180

In [2]:
angle = torch.tensor([   0,  90,   0,   0,  45, 130,  45,  90,   0,  22,  90,  90,  45,    0])
offset = torch.tensor([100, 125, 110, 580, 755, 770, 120, 500, 975, 240, 450, 520, 200, 1000])
angle = angle * torch.pi/180
angle, offset

(tensor([0.0000, 1.5708, 0.0000, 0.0000, 0.7854, 2.2689, 0.7854, 1.5708, 0.0000,
         0.3840, 1.5708, 1.5708, 0.7854, 0.0000]),
 tensor([ 100,  125,  110,  580,  755,  770,  120,  500,  975,  240,  450,  520,
          200, 1000]))

In [3]:
angle_bin_index = torch.floor(angle / bin_angle_size).to(torch.int32)
offset_bin_index = torch.floor(offset / bin_offset_size).to(torch.int32)
offset_bin_index -= offset_bin_index.min()
angle_bin_index, offset_bin_index

(tensor([  0, 360,   0,   0, 180, 520, 180, 360,   0,  88, 360, 360, 180,   0],
        dtype=torch.int32),
 tensor([ 0,  1,  0, 19, 26, 26,  0, 16, 35,  5, 14, 16,  4, 36],
        dtype=torch.int32))

In [4]:
bin_hash = angle_bin_index * (offset_bin_index.max() + 1) + offset_bin_index
bin_hash

tensor([    0, 13321,     0,    19,  6686, 19266,  6660, 13336,    35,  3261,
        13334, 13336,  6664,    36], dtype=torch.int32)

In [5]:
sorted_hash, original_index = torch.sort(bin_hash)
sorted_hash, original_index

(tensor([    0,     0,    19,    35,    36,  3261,  6660,  6664,  6686, 13321,
         13334, 13336, 13336, 19266], dtype=torch.int32),
 tensor([ 0,  2,  3,  8, 13,  9,  6, 12,  4,  1, 10,  7, 11,  5]))

In [7]:
mask = torch.hstack([torch.tensor([True]), sorted_hash[1:]!=sorted_hash[:-1]])
mask

tensor([ True, False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True, False,  True])

In [8]:
bin_indices = sorted_hash[mask]
bin_indices

tensor([    0,    19,    35,    36,  3261,  6660,  6664,  6686, 13321, 13334,
        13336, 19266], dtype=torch.int32)

In [35]:
angle_bin = bin_indices // (offset_bin_index.max() + 1)
offset_bin = bin_indices % (offset_bin_index.max() + 1)
angle_bin, offset_bin

(tensor([  0,   0,   0,   0,  88, 180, 180, 180, 360, 360, 360, 520],
        dtype=torch.int32),
 tensor([ 0, 19, 35, 36,  5,  0,  4, 26,  1, 14, 16, 26], dtype=torch.int32))

In [47]:
bin_start = torch.where(mask)[0]
bin_end = torch.hstack([bin_start[1:], torch.tensor([len(sorted_hash)])])
bin_start, bin_end

(tensor([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13]),
 tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14]))

In [36]:
bin_index_map = {int(h.item()): i for i, h in enumerate(bin_indices)}
bin_index_map

{0: 0,
 19: 1,
 35: 2,
 36: 3,
 3261: 4,
 6660: 5,
 6664: 6,
 6686: 7,
 13321: 8,
 13334: 9,
 13336: 10,
 19266: 11}

In [37]:
[k for k in range(len(bin_indices))]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [46]:
angle_i = angle_bin[10].item()
offset_j = offset_bin[10].item()
current_hash = bin_indices[10].item()
angle_i, offset_j, current_hash

(360, 16, 13336)